In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random 
from random import randint

# Exercise 1.1 

In [142]:
# To compute individual expected value of each arm 
arms = np.array([[2,3],[-1,5],[1,5],[-2,4],[0,3],[2,6]])
expected_value_each_arm = np.mean(arms,axis=1)
expected_value_first_arm = expected_value_each_arm[0]
expected_value_second_arm = expected_value_each_arm[1]
expected_value_third_arm = expected_value_each_arm[2]
expected_value_fourth_arm = expected_value_each_arm[3]
expected_value_fifth_arm = expected_value_each_arm[4]
expected_value_sixth_arm = expected_value_each_arm[5]

cumulative_expected_value = expected_value_first_arm +  expected_value_second_arm + expected_value_third_arm+ expected_value_fourth_arm+ expected_value_fifth_arm + expected_value_sixth_arm

In [141]:
print 'expected_value arm1',expected_value_first_arm
print 'expected_value arm2', expected_value_second_arm
print 'expected_value arm3', expected_value_third_arm
print 'expected_value arm4', expected_value_fourth_arm
print 'expected_value arm5',expected_value_fifth_arm
print 'expected_value arm6', expected_value_sixth_arm
print 'cumulative expected value',cumulative_expected_value

expected_value arm1 2.5
expected_value arm2 2.0
expected_value arm3 3.0
expected_value arm4 1.0
expected_value arm5 1.5
expected_value arm6 4.0
cumulative expected value 14.0


# Exercise 1.2

In [143]:
number_of_arms = 6
action_rewards = np.array([[np.random.uniform(2,3)],[np.random.uniform(-1,5)],[np.random.uniform(1,5)],[np.random.uniform(-2,4)],
                [np.random.uniform(0,3)],[np.random.uniform(2,6)]])
action_value_estimate = np.zeros(number_of_arms)
iterations = 100
count_occurance_of_action = np.zeros(number_of_arms)
reward_of_selected_arm = np.zeros(number_of_arms)
for i in range(iterations):
    select_arm = np.random.uniform(0,6)
    reward_of_selected_arm = action_rewards[select_arm]
    count_occurance_of_action[select_arm]+=1
    action_value_estimate[select_arm]+= (reward_of_selected_arm)/count_occurance_of_action[select_arm] 
    
best_estimate = np.argmax(action_value_estimate) 

/home/ramesh/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/ramesh/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:11: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/ramesh/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [135]:
class Armed_Bandits(object):
    def __init__(self, bandits, Q):
        self.bandits = bandits
        self.Q = Q
        self.arms = len(bandits[:,0])
        
        
        self.optimistic_Q = np.array([[0.0, 0],
             [0.0, 0],
             [0.0, 0],
             [0.0, 0],
             [0.0, 0],
             [0.0, 0]])

    # Exercise 1.2
    
    def sample_bandit(self, bandit_index):
        expected_value = np.random.uniform(self.bandits[bandit_index,0],self.bandits[bandit_index,1])
        return expected_value
    
    def average_reward(self,iterations):
        rewards = []
        for i in range(iterations):
            bandit_index = np.random.randint(0,self.arms)
            sample = [bandit_index, self.sample_bandit(bandit_index)]
            rewards.append(sample)
        average_rewards = np.mean(np.array(rewards)[:,1])
        print('Average reward = ' + str(average_rewards))
    
    # Exercise 1.3 
    
    def print_Q(self):
        for i in range(self.arms):
            print('Arm ' + str(i+1) + ' = ' + str(100*self.Q[i,1]/np.sum(self.Q[:,1])) + '%')
        
    def e_greedy(self, iterations, frequency, e):
        rewards = []
        for i in range(iterations):
            if np.random.random() > e:
                bandit_index = np.max(self.Q[:,0])
            else:
                bandit_index = np.random.randint(0,self.arms)
            average_reward = self.sample_bandit(bandit_index)
            sample = [bandit_index, average_reward]
            rewards.append(sample)
            
           
            self.Q[bandit_index,1] = self.Q[bandit_index,1] + 1
            self.Q[bandit_index,0] = self.Q[bandit_index,0] + (average_reward - self.Q[bandit_index,0])/self.Q[bandit_index,1]
            if ((i + 1) % frequency) == 0:
                self.print_Q()
                print('Average reward = ' + str(np.mean(np.array(rewards)[:,1])))
                
        
    def third_arm_uniformly(self,iterations,frequency,e,alpha):
        rewards = []
        k = iterations
        summation = 0
        for i in range(iterations):
            
            if np.random.random() > e:
                bandit_index = np.max(self.Q[:,0])
            else:
                bandit_index = np.random.randint(0,self.arms)
            average_reward = self.sample_bandit(bandit_index)
            sample = [bandit_index, average_reward]
            rewards.append(sample)
            
            
            summation = alpha*((1-alpha)**(k-i))*average_reward + summation 
            self.Q[bandit_index,0] = self.Q[0,0]*((1 - alpha)**k) + summation 
            
            # after 500 iterations
            
            if i > 500:
                self.bandits[2] = np.array([6,8])
            if ((i + 1) % frequency) == 0:
                self.print_Q()
                print('Average reward = ' + str(np.mean(np.array(rewards)[:,1])))
                
            #plt.figure()
            #plt.plot(range(0,iterations),np.array(rewards)[:,0])
            #plt.show()
    def optimistic_intiallization(self,iterations,frequency,alpha):
        rewards = []
        k = iterations
        summation = 0
        
        for i in range(iterations):
            bandit_index = np.max(self.optimistic_Q[:,0])
            average_reward = self.sample_bandit(bandit_index)
            sample = [bandit_index, average_reward]
            rewards.append(sample)
            
        summation = alpha*((1-alpha)**(k-i))*average_reward + summation 
        self.optimistic_Q[bandit_index,0] = self.optimistic_Q[0,0]*((1 - alpha)**k) + summation 
        
        if i > 500:
            self.bandits[2] = np.array([6,8])
        if ((i + 1) % frequency) == 0:
            self.print_Q()
            print('Average reward = ' + str(np.mean(np.array(rewards)[:,1])))
        
        
    
    

In [136]:
reward_distribution = np.array([[2,3],
                                [-1,5],
                                [1,5],
                                [-2,4],
                                [0,3],
                                [2,6]])
Q = np.array([[0.0, 0],
             [0.0, 0],
             [0.0, 0],
             [0.0, 0],
             [0.0, 0],
             [0.0, 0]])
bandits = Armed_Bandits(reward_distribution, Q)
bandits.average_reward(100)

Average reward = 2.30066248821


In [137]:
bandits.e_greedy(1000,100,0.1)

Arm 1 = 2.0%
Arm 2 = 1.0%
Arm 3 = 1.0%
Arm 4 = 2.0%
Arm 5 = 89.0%
Arm 6 = 5.0%
Average reward = 1.75296280559
Arm 1 = 2.5%
Arm 2 = 1.0%
Arm 3 = 1.5%
Arm 4 = 3.0%
Arm 5 = 89.0%
Arm 6 = 3.0%
Average reward = 1.49425825498
Arm 1 = 1.66666666667%
Arm 2 = 1.66666666667%
Arm 3 = 1.33333333333%
Arm 4 = 2.33333333333%
Arm 5 = 90.3333333333%
Arm 6 = 2.66666666667%
Average reward = 1.52687938108
Arm 1 = 1.5%
Arm 2 = 1.25%
Arm 3 = 1.25%
Arm 4 = 2.0%
Arm 5 = 92.0%
Arm 6 = 2.0%
Average reward = 1.51292697608
Arm 1 = 1.4%
Arm 2 = 1.2%
Arm 3 = 1.8%
Arm 4 = 2.2%
Arm 5 = 91.2%
Arm 6 = 2.2%
Average reward = 1.53810181493
Arm 1 = 1.66666666667%
Arm 2 = 1.16666666667%
Arm 3 = 1.66666666667%
Arm 4 = 2.33333333333%
Arm 5 = 91.1666666667%
Arm 6 = 2.0%
Average reward = 1.53152361354
Arm 1 = 1.57142857143%
Arm 2 = 1.28571428571%
Arm 3 = 1.85714285714%
Arm 4 = 2.42857142857%
Arm 5 = 90.8571428571%
Arm 6 = 2.0%
Average reward = 1.54699926224
Arm 1 = 1.625%
Arm 2 = 1.125%
Arm 3 = 1.75%
Arm 4 = 2.125%
Arm 5 = 91.3

/home/ramesh/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:18: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/ramesh/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:48: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/ramesh/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:49: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [138]:
alpha = 0.01
bandits.third_arm_uniformly(1000,100,0.1,alpha)

Arm 1 = 1.8%
Arm 2 = 1.1%
Arm 3 = 1.6%
Arm 4 = 2.0%
Arm 5 = 91.6%
Arm 6 = 1.9%
Average reward = 1.71684000869
Arm 1 = 1.8%
Arm 2 = 1.1%
Arm 3 = 1.6%
Arm 4 = 2.0%
Arm 5 = 91.6%
Arm 6 = 1.9%
Average reward = 2.12325273439
Arm 1 = 1.8%
Arm 2 = 1.1%
Arm 3 = 1.6%
Arm 4 = 2.0%
Arm 5 = 91.6%
Arm 6 = 1.9%
Average reward = 2.2769288664
Arm 1 = 1.8%
Arm 2 = 1.1%
Arm 3 = 1.6%
Arm 4 = 2.0%
Arm 5 = 91.6%
Arm 6 = 1.9%
Average reward = 2.33915813766
Arm 1 = 1.8%
Arm 2 = 1.1%
Arm 3 = 1.6%
Arm 4 = 2.0%
Arm 5 = 91.6%
Arm 6 = 1.9%
Average reward = 2.36269705612
Arm 1 = 1.8%
Arm 2 = 1.1%
Arm 3 = 1.6%
Arm 4 = 2.0%
Arm 5 = 91.6%
Arm 6 = 1.9%
Average reward = 2.41063977057
Arm 1 = 1.8%
Arm 2 = 1.1%
Arm 3 = 1.6%
Arm 4 = 2.0%
Arm 5 = 91.6%
Arm 6 = 1.9%
Average reward = 2.43072844824
Arm 1 = 1.8%
Arm 2 = 1.1%
Arm 3 = 1.6%
Arm 4 = 2.0%
Arm 5 = 91.6%
Arm 6 = 1.9%
Average reward = 2.45437241073
Arm 1 = 1.8%
Arm 2 = 1.1%
Arm 3 = 1.6%
Arm 4 = 2.0%
Arm 5 = 91.6%
Arm 6 = 1.9%
Average reward = 2.4794966646
Arm 1 = 1.8%

/home/ramesh/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:18: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/ramesh/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:71: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [139]:
bandits.optimistic_intiallization(1000,100,alpha)

/home/ramesh/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:18: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


IndexError: index 10 is out of bounds for axis 0 with size 6